## Webscraping notebook

In [79]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [80]:
#loading downloaded data
df=pd.read_json('~/Downloads/reviews_Electronics_5.json', lines=True)

In [81]:
#examining text of reviews
df.reviewText

0          We got this GPS for my husband who is an (OTR)...
1          I'm a professional OTR truck driver, and I bou...
2          Well, what can I say.  I've had this unit in m...
3          Not going to write a long review, even thought...
4          I've had mine for a year and here's what we go...
                                 ...                        
1689183    Burned these in before listening to them for a...
1689184    Some people like DJ style headphones or earbud...
1689185    I&#8217;m a big fan of the Brainwavz S1 (actua...
1689186    I've used theBrainwavz S1 In Ear Headphones, a...
1689187    Normally when I receive a review sample I can ...
Name: reviewText, Length: 1689188, dtype: object

In [4]:
#error asins
# df= df[df.asin!='B00004SYNX']
# df= df[df.asin!='B00000J1SC']
# df= df[df.asin!='B000067RVL']
# df= df[df.asin!='B000068O53']
# df= df[df.asin!='B00006B8EC']
# df= df[df.asin!='']

### Webscraping the urls productTitle for item names

In [51]:
# grabbing only the first 10,000 unique asin 
asin_list= list(df.asin.unique())
asin_list=asin_list[0:10000]
asin_list1=asin_list[10000:20000]
asin_list2=asin_list[20000:30000]

In [61]:
asin_list= list(df.asin.unique())
asin_list1=asin_list[10000:20000]

In [6]:
#getting urls of asin's 
url=['https://www.amazon.com/dp/{}'.format(x) for x in asin_list]

In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

options = Options()
options.add_argument("start-maximized")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument('--disable-gpu')
options.headless = True
options.add_argument("window-size=1920,1080")
driver = webdriver.Chrome(chrome_options=options)

#initializing selenium
driver = webdriver.Chrome()

/var/folders/48/3rfqt3q17zj4zsvn_9rh50br0000gn/T/ipykernel_54878/3977662427.py:11: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=options)


In [8]:
#script with selenium driver to extract element with productname
import time
stuff=[]
links=[]
for x in url:
    try:
        driver.get(str(x))
        content=driver.find_element_by_id('productTitle')
        links.append(x)
        stuff.append(content.text)
    except:
        print('page not found')
        continue      
        time.sleep(.001)

/var/folders/48/3rfqt3q17zj4zsvn_9rh50br0000gn/T/ipykernel_54878/3325878413.py:8: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  content=driver.find_element_by_id('productTitle')


In [56]:
import pickle
with open ('stuff.pickle', 'wb') as f:
    pickle.dump(stuff, f, 5)


In [57]:
with open ('links.pickle', 'wb') as f:
    pickle.dump(links, f, 5)

In [24]:
len(links)

9423

In [25]:
len(stuff)

9423

In [63]:
#getting urls of asin's
url1=['https://www.amazon.com/dp/{}'.format(x) for x in asin_list1]

In [64]:
asin_list1[0]

'B000IMM26U'

In [65]:
#initializing selenium
driver = webdriver.Chrome()

In [66]:
#script with selenium driver to extract element with productname using for loop

import time
stuff1=[]
links1=[]
for x in tqdm(url1):
    try:
        driver.get(str(x))
        content=driver.find_element_by_id('productTitle')
        links1.append(x)
        stuff1.append(content.text)
    except:
        continue      
        time.sleep(.001)

  0%|                                                 | 0/10000 [00:00<?, ?it/s]/var/folders/48/3rfqt3q17zj4zsvn_9rh50br0000gn/T/ipykernel_54878/1158868331.py:9: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  content=driver.find_element_by_id('productTitle')
100%|███████████████████████████████████| 10000/10000 [5:15:51<00:00,  1.90s/it]


In [67]:

with open ('stuff1.pickle', 'wb') as f:
    pickle.dump(stuff1, f, 5)
with open ('links1.pickle', 'wb') as f:
    pickle.dump(links1, f, 5)

In [68]:
asin_list= list(df.asin.unique())
asin_list2=asin_list[20000:30000]

In [69]:
url2=['https://www.amazon.com/dp/{}'.format(x) for x in asin_list2]

In [70]:
options = Options()
options.add_argument("start-maximized")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument('--disable-gpu')
options.headless = True
options.add_argument("window-size=1920,1080")
driver = webdriver.Chrome(chrome_options=options)

#initializing selenium
driver = webdriver.Chrome()

/var/folders/48/3rfqt3q17zj4zsvn_9rh50br0000gn/T/ipykernel_54878/1509041771.py:8: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=options)


In [71]:
#script with selenium driver to extract element with productname
import time
from tqdm import tqdm

stuff2=[]
links2=[]
for x in tqdm(url2):
    try:
        driver.get(str(x))
        content=driver.find_element_by_id('productTitle')
        links2.append(x)
        stuff2.append(content.text)
    except:
        continue      
        time.sleep(.0005)

  0%|                                                 | 0/10000 [00:00<?, ?it/s]/var/folders/48/3rfqt3q17zj4zsvn_9rh50br0000gn/T/ipykernel_54878/2029006198.py:10: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  content=driver.find_element_by_id('productTitle')
100%|███████████████████████████████████| 10000/10000 [5:19:08<00:00,  1.91s/it]


In [75]:
import pickle
with open ('stuff2.pickle', 'wb') as f:
    pickle.dump(stuff2, f, 5)
with open ('links2.pickle', 'wb') as f:
    pickle.dump(links2, f, 5)

### Webscraping the item tags so that items can be put into categories

In [125]:
#getting our urls ready for webscraping
df1= pd.Series(links)
df2=pd.Series(links1)
df3=pd.Series(links2)
frames=[df1,df2,df3]
url_series=pd.concat(frames, axis=0)
len(url_series)

26885

In [52]:
#importing webdriver
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

# options = Options()
# options.add_argument("start-maximized")
# options.add_experimental_option("excludeSwitches", ["enable-automation"])
# options.add_experimental_option('useAutomationExtension', False)
# options.add_argument('--disable-gpu')
# options.headless = True
# options.add_argument("window-size=1920,1080")
# driver = webdriver.Chrome(chrome_options=options)

#initializing selenium
driver = webdriver.Chrome()

/var/folders/48/3rfqt3q17zj4zsvn_9rh50br0000gn/T/ipykernel_1871/3977662427.py:11: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=options)


In [129]:
#script to scrape the categories from items
from tqdm import tqdm
import time
linkz=[]
tagz=[]
driver = webdriver.Chrome()
for x in tqdm(url_series):
    try:
        driver.get(str(x))
        content=driver.find_elements_by_class_name('a-link-normal.a-color-tertiary')
        # see= [x.text for x in content]
        # tagz.append(see)
        linkz.append(x)
    except:
        continue
        time.sleep(.0005)
        driver.close()

  0%|                                                 | 0/26885 [00:00<?, ?it/s]/var/folders/48/3rfqt3q17zj4zsvn_9rh50br0000gn/T/ipykernel_1871/3385348802.py:10: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  content=driver.find_elements_by_class_name('a-link-normal.a-color-tertiary')
100%|██████████████████████████████████| 26885/26885 [17:20:37<00:00,  2.32s/it]


100%|████████████████████████████████| 26871/26871 [00:00<00:00, 4266063.92it/s]


In [141]:
#these category names will serve as tags for our items
tagzz=[]
for ele in tqdm(tagz):
    tagzz.append(ele[-1::])

    

100%|████████████████████████████████| 26871/26871 [00:00<00:00, 1616887.49it/s]


In [144]:
import pickle
with open ('tagzz.pickle', 'wb') as f:
    pickle.dump(tagzz, f, 5)

### Scraping the price of the item

In [ ]:
from tqdm import tqdm
import time
linkz_p=[]
tagz_p=[]
driver = webdriver.Chrome()
for x in tqdm(url_series):
    try:
        driver.get(str(x))
        content=driver.find_elements_by_class_name('a-price')
        see= [x.text for x in content]
        tagz_p.append(see)
        linkz_p.append(x)
    except:
        continue
        time.sleep(.0005)
        driver.close()

  0%|                                                 | 0/26885 [00:00<?, ?it/s]/var/folders/48/3rfqt3q17zj4zsvn_9rh50br0000gn/T/ipykernel_1871/717161247.py:9: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  content=driver.find_elements_by_class_name('a-price')
 72%|███████████████████████         | 19399/26885 [13:07:42<3:55:04,  1.88s/it]

### We will use Bert Embeddings for the df.reviewText

In [156]:
len(linkz_p), len(tagzz)

['https://www.amazon.com/dp/0528881469',
 'https://www.amazon.com/dp/0594451647',
 'https://www.amazon.com/dp/0594481813',
 'https://www.amazon.com/dp/0972683275',
 'https://www.amazon.com/dp/1400501466']

In [159]:
tagzz[0:5]

[['Trucking GPS'],
 ['Chargers & Adapters'],
 ['Power Adapters'],
 ['TV Ceiling & Wall Mounts'],
 ['Tablets']]